In [1]:
import os, time, duckdb, torch, timm, gc
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier
from sklearn.linear_model import SGDClassifier
from PIL import Image
from sklearn.model_selection import ParameterGrid

from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import f1_score, accuracy_score, classification_report, confusion_matrix
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier

import torchvision.transforms as T
from pathlib import Path

from torch.utils.data import Dataset, DataLoader
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
import duckdb, torch
from transformers import CLIPModel, CLIPProcessor

In [2]:
DB_PATH = "D:/db/meta.duckdb"
con = duckdb.connect(DB_PATH)
try:
    con.execute("PRAGMA threads=8;")
except duckdb.InvalidInputException:
    pass

print("Set up ready")

Set up ready


# LOAD EMBEDDINGS

In [2]:
train = np.load("D:/dataset/clip_img_emb_ALL/clip_vit_b32_train_ALL.npz", allow_pickle = True)

X_tr = train["feats"]
ids_tr = train["post_id"]

print(X_tr.shape, len(ids_tr))

(960048, 512) 960048


In [3]:
df = pd.DataFrame({
    "post_id": ids_tr
})

df["emb"] = list(X_tr)

agg = df.groupby("post_id")["emb"].apply(lambda x: np.mean(x.tolist(), axis=0))

X_tr = np.stack(agg.values)
post_ids_unique = agg.index.values

In [4]:
ids_list = post_ids_unique.tolist()

DB_PATH = "D:/db/meta.duckdb"
con = duckdb.connect(DB_PATH)
try:
    con.execute("PRAGMA threads=8;")
except duckdb.InvalidInputException:
    pass

print("Set up ready")

con.execute("CREATE TEMP TABLE tmp_ids (post_id VARCHAR, ord INT)")

con.execute(
    "INSERT INTO tmp_ids VALUES " +
    ", ".join(f"('{pid}', {i})" for i, pid in enumerate(ids_list))
)

targets = con.execute("""
    SELECT md1718.er_bins3
    FROM md1718
    JOIN tmp_ids USING(post_id)
    ORDER BY tmp_ids.ord
""").fetchall()

y_tr = np.array([t[0] for t in targets])


Set up ready


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [5]:
np.save("D:/dataset/clip_img_emb_ALL/y_tr_3.npy", y_tr)

In [6]:
del X_tr, ids_tr, y_tr, train, df, agg, post_ids_unique, ids_list, targets
gc.collect()

0

In [7]:
val = np.load("D:/dataset/clip_img_emb_ALL/clip_vit_b32_validation_ALL.npz", allow_pickle = True)

X_va = val["feats"]
ids_va = val["post_id"]

print(X_va.shape, len(ids_va))

(556982, 512) 556982


In [8]:
df = pd.DataFrame({
    "post_id": ids_va
})

df["emb"] = list(X_va)

agg = df.groupby("post_id")["emb"].apply(lambda x: np.mean(x.tolist(), axis=0))

X_va = np.stack(agg.values)
post_ids_unique_va = agg.index.values

In [9]:
ids_list = post_ids_unique_va.tolist()

DB_PATH = "D:/db/meta.duckdb"
con = duckdb.connect(DB_PATH)
try:
    con.execute("PRAGMA threads=8;")
except duckdb.InvalidInputException:
    pass

print("Set up ready")

con.execute("CREATE TEMP TABLE tmp_ids (post_id VARCHAR, ord INT)")

con.execute(
    "INSERT INTO tmp_ids VALUES " +
    ", ".join(f"('{pid}', {i})" for i, pid in enumerate(ids_list))
)

targets = con.execute("""
    SELECT md1718.er_bins3
    FROM md1718
    JOIN tmp_ids USING(post_id)
    ORDER BY tmp_ids.ord
""").fetchall()

y_va = np.array([t[0] for t in targets])


Set up ready


In [10]:
print(len(y_va), len(ids_list))

412325 412325


In [11]:
np.save("D:/dataset/clip_img_emb_ALL/y_va_3.npy", y_va)

In [ ]:
del X_va, ids_va, y_va, val, df, agg, post_ids_unique_va, ids_list, targets
gc.collect()

In [13]:
del  post_ids_unique_va, ids_list, targets
gc.collect()

479

In [2]:
X_tr = np.load("D:/dataset/clip_img_emb_ALL/X_tr.npy", allow_pickle = True)
X_va = np.load("D:/dataset/clip_img_emb_ALL/X_va.npy", allow_pickle = True)

y_tr = np.load("D:/dataset/clip_img_emb_ALL/y_tr_3.npy", allow_pickle = True)
y_va = np.load("D:/dataset/clip_img_emb_ALL/y_va_3.npy", allow_pickle = True)

In [21]:
# SGD
param_grid = {
    "alpha": [1e-5, 1e-4, 1e-3, 1e-2],
    "class_weight": [None, "balanced"],
}

results = []
best_score = -np.inf
best_params = None


for params in ParameterGrid(param_grid):
    print(f"\nCombination: {params}")

    clf = SGDClassifier(
        loss="hinge",            
        penalty="l2",            
        **params,
        average = True,
        random_state=42,
        max_iter=1000,
        tol=1e-3,
    )

    clf.fit(X_tr, y_tr)

    y_val_pred = clf.predict(X_va)

    macro_f1 = f1_score(y_va, y_val_pred, average="macro")
    acc = accuracy_score(y_va, y_val_pred)

    print(f"macro-F1 (val): {macro_f1} | accuracy (val): {acc}")

    results.append({
        "alpha": params["alpha"],
        "class_weight": params["class_weight"],
        "val_macro_f1": macro_f1,
        "val_accuracy": acc,
    })

    if macro_f1 > best_score:
        best_score = macro_f1
        best_params = params

print("\nBest hyperparameter configuration:")
print(best_params)
print("Validation macro-F1:", best_score)

results_df = pd.DataFrame(results).sort_values("val_macro_f1", ascending=False)
print("\nOrdered results by macro-F1 (validation):")
print(results_df)


Combination: {'alpha': 1e-05, 'class_weight': None}
macro-F1 (val): 0.41889013603906217 | accuracy (val): 0.44565573273509973

Combination: {'alpha': 1e-05, 'class_weight': 'balanced'}
macro-F1 (val): 0.40264573282624827 | accuracy (val): 0.44534529800521433

Combination: {'alpha': 0.0001, 'class_weight': None}
macro-F1 (val): 0.40707643087852247 | accuracy (val): 0.4414939671375735

Combination: {'alpha': 0.0001, 'class_weight': 'balanced'}
macro-F1 (val): 0.3930324530098044 | accuracy (val): 0.43990541441823805

Combination: {'alpha': 0.001, 'class_weight': None}
macro-F1 (val): 0.40852275446450786 | accuracy (val): 0.44088764930576607

Combination: {'alpha': 0.001, 'class_weight': 'balanced'}
macro-F1 (val): 0.39017696811245717 | accuracy (val): 0.43923361426059543

Combination: {'alpha': 0.01, 'class_weight': None}
macro-F1 (val): 0.41223532946945124 | accuracy (val): 0.44126114109015946

Combination: {'alpha': 0.01, 'class_weight': 'balanced'}
macro-F1 (val): 0.39099568885249564 

In [3]:
# NAIVE BAYES - GAUSSIAN
param_grid_nb = {
    "var_smoothing": [1e-9, 1e-8, 1e-7, 1e-6]
}

results = []
best_score = -np.inf
best_params = None

for params in ParameterGrid(param_grid_nb):
    print(f"\nCombination: {params}")

    clf = GaussianNB(**params)

    # Fit su TRAIN
    clf.fit(X_tr, y_tr)

    # Valutazione su VALIDATION
    y_val_pred = clf.predict(X_va)

    macro_f1 = f1_score(y_va, y_val_pred, average="macro")
    acc = accuracy_score(y_va, y_val_pred)

    print(f"macro-F1 (val): {macro_f1:.4f} | accuracy (val): {acc:.4f}")

    results.append({
        "var_smoothing": params["var_smoothing"],
        "val_macro_f1": macro_f1,
        "val_accuracy": acc,
    })

    # Aggiorno il best model in base alla macro-F1
    if macro_f1 > best_score:
        best_score = macro_f1
        best_params = params

print("\nBest hyperparameter configuration:")
print(best_params)
print("Validation macro-F1:", best_score)

# Metto i risultati in un DataFrame per ispezionarli meglio
results_df = pd.DataFrame(results).sort_values("val_macro_f1", ascending=False)
print("\nOrdered results by macro-F1 (validation):")
print(results_df)


Combination: {'var_smoothing': 1e-09}
macro-F1 (val): 0.3973 | accuracy (val): 0.4314

Combination: {'var_smoothing': 1e-08}
macro-F1 (val): 0.3973 | accuracy (val): 0.4314

Combination: {'var_smoothing': 1e-07}
macro-F1 (val): 0.3973 | accuracy (val): 0.4314

Combination: {'var_smoothing': 1e-06}
macro-F1 (val): 0.3973 | accuracy (val): 0.4314

Best hyperparameter configuration:
{'var_smoothing': 1e-06}
Validation macro-F1: 0.39729505839467083

Ordered results by macro-F1 (validation):
   var_smoothing  val_macro_f1  val_accuracy
3   1.000000e-06      0.397295      0.431354
0   1.000000e-09      0.397294      0.431354
1   1.000000e-08      0.397292      0.431351
2   1.000000e-07      0.397292      0.431351


In [4]:
# RANDOM FOREST
param_grid_rf = {
    "n_estimators": [50, 80],
    "max_depth": [10, 12], 
    "min_samples_leaf": [2, 5],       
    "max_features": [0.05, "sqrt"],   
}

results = []
best_score = -np.inf
best_params = None

for params in ParameterGrid(param_grid_rf):
    print(f"\nCombination: {params}")

    clf = RandomForestClassifier(
        **params,
        n_jobs=-1,
        random_state=42
    )

    # Fit su TRAIN
    clf.fit(X_tr, y_tr)

    # Valutazione su VALIDATION
    y_val_pred = clf.predict(X_va)

    macro_f1 = f1_score(y_va, y_val_pred, average="macro")
    acc = accuracy_score(y_va, y_val_pred)

    print(f"macro-F1 (val): {macro_f1:.4f} | accuracy (val): {acc:.4f}")

    results.append({
        "n_estimators": params["n_estimators"],
        "max_depth": params["max_depth"],
        "min_samples_leaf": params["min_samples_leaf"],
        "max_features": params["max_features"],
        "val_macro_f1": macro_f1,
        "val_accuracy": acc,
    })

    # Aggiorno il best model in base alla macro-F1
    if macro_f1 > best_score:
        best_score = macro_f1
        best_params = params

print("\nBest hyperparameter configuration (Random Forest):")
print(best_params)
print("Validation macro-F1:", best_score)

# Metto i risultati in un DataFrame per ispezionarli meglio
results_df_rf = pd.DataFrame(results).sort_values("val_macro_f1", ascending=False)
print("\nOrdered results by macro-F1 (validation):")
print(results_df_rf)



Combination: {'max_depth': 10, 'max_features': 0.05, 'min_samples_leaf': 2, 'n_estimators': 50}
macro-F1 (val): 0.4322 | accuracy (val): 0.4313

Combination: {'max_depth': 10, 'max_features': 0.05, 'min_samples_leaf': 2, 'n_estimators': 80}
macro-F1 (val): 0.4333 | accuracy (val): 0.4324

Combination: {'max_depth': 10, 'max_features': 0.05, 'min_samples_leaf': 5, 'n_estimators': 50}
macro-F1 (val): 0.4325 | accuracy (val): 0.4316

Combination: {'max_depth': 10, 'max_features': 0.05, 'min_samples_leaf': 5, 'n_estimators': 80}
macro-F1 (val): 0.4337 | accuracy (val): 0.4328

Combination: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'n_estimators': 50}
macro-F1 (val): 0.4325 | accuracy (val): 0.4318

Combination: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'n_estimators': 80}
macro-F1 (val): 0.4335 | accuracy (val): 0.4326

Combination: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 5, 'n_estimators': 50}
macro-F1 (val): 0.4325 | ac

In [5]:
# XGBOOST

# Convert the labels into numbers
le = LabelEncoder()
y_tr_enc = le.fit_transform(y_tr)
y_val_enc = le.transform(y_va)


param_grid_xgb = {
    "n_estimators": [100, 150], 
    "max_depth": [4, 6],        
    "learning_rate": [0.1],     
    "subsample": [0.8],         
    "colsample_bytree": [0.5],  
    "gamma": [0, 1],            
    "reg_lambda": [1],          
}

results = []
best_score = -np.inf
best_params = None

for params in ParameterGrid(param_grid_xgb):
    print(f"\nCombination: {params}")

    clf = XGBClassifier(
        **params,
        objective="multi:softmax",
        num_class=len(np.unique(y_tr_enc)),
        tree_method="hist",
        eval_metric="mlogloss",
        n_jobs=-1,
        random_state=42,
        verbosity=0,
    )

    # Fit
    clf.fit(X_tr, y_tr_enc)

    # Validation
    y_val_pred = clf.predict(X_va)

    macro_f1 = f1_score(y_val_enc, y_val_pred, average="macro")
    acc = accuracy_score(y_val_enc, y_val_pred)

    print(f"macro-F1 (val): {macro_f1:.4f} | accuracy (val): {acc:.4f}")

    results.append({
        **params,
        "val_macro_f1": macro_f1,
        "val_accuracy": acc,
    })

    if macro_f1 > best_score:
        best_score = macro_f1
        best_params = params

print("\nBest hyperparameter configuration (XGBoost):")
print(best_params)
print("Validation macro-F1:", best_score)

results_df_xgb = pd.DataFrame(results).sort_values("val_macro_f1", ascending=False)
print("\nOrdered results:")
print(results_df_xgb)


Combination: {'colsample_bytree': 0.5, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'reg_lambda': 1, 'subsample': 0.8}
macro-F1 (val): 0.4375 | accuracy (val): 0.4407

Combination: {'colsample_bytree': 0.5, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 150, 'reg_lambda': 1, 'subsample': 0.8}
macro-F1 (val): 0.4398 | accuracy (val): 0.4431

Combination: {'colsample_bytree': 0.5, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 100, 'reg_lambda': 1, 'subsample': 0.8}
macro-F1 (val): 0.4414 | accuracy (val): 0.4446

Combination: {'colsample_bytree': 0.5, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 150, 'reg_lambda': 1, 'subsample': 0.8}
macro-F1 (val): 0.4439 | accuracy (val): 0.4469

Combination: {'colsample_bytree': 0.5, 'gamma': 1, 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'reg_lambda': 1, 'subsample': 0.8}
macro-F1 (val): 0.4375 | accuracy (val): 0.4407

Combination: {'colsample_bytr

# PERFORMANCE TEST SET

In [3]:
test = np.load("D:/dataset/clip_img_emb_ALL/clip_vit_b32_test_ALL.npz", allow_pickle = True)

# accedi alle chiavi con le stringhe tra virgolette
X_te = test["feats"]
ids_te = test["post_id"]

print(X_te.shape, len(ids_te))

(588557, 512) 588557


In [4]:
df = pd.DataFrame({
    "post_id": ids_te
})

# Inseriamo gli embeddings in un array di oggetti
df["emb"] = list(X_te)

# Aggrega per post_id
agg = df.groupby("post_id")["emb"].apply(lambda x: np.mean(x.tolist(), axis=0))

X_te = np.stack(agg.values)
post_ids_unique_te = agg.index.values

np.save("D:/dataset/clip_img_emb_ALL/X_te.npy", X_te)

In [5]:
ids_list = post_ids_unique_te.tolist()

con.execute("CREATE TEMP TABLE tmp_ids (post_id VARCHAR, ord INT)")

con.execute(
    "INSERT INTO tmp_ids VALUES " +
    ", ".join(f"('{pid}', {i})" for i, pid in enumerate(ids_list))
)

targets = con.execute("""
    SELECT md1718.er_bins3
    FROM md1718
    JOIN tmp_ids USING(post_id)
    ORDER BY tmp_ids.ord
""").fetchall()

y_te = np.array([t[0] for t in targets])


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [6]:
print(len(y_te), len(ids_list))

423604 423604


In [7]:
np.save("D:/dataset/clip_img_emb_ALL/y_te_3.npy", y_te)

In [8]:
del X_te, ids_te, y_te, test, df, agg, post_ids_unique_te, ids_list, targets
gc.collect()

0

In [4]:
X_tr = np.load("D:/dataset/clip_img_emb_ALL/X_tr.npy", allow_pickle = True)
y_tr = np.load("D:/dataset/clip_img_emb_ALL/y_tr_3.npy", allow_pickle = True)

X_va = np.load("D:/dataset/clip_img_emb_ALL/X_va.npy", allow_pickle = True)
y_va = np.load("D:/dataset/clip_img_emb_ALL/y_va_3.npy", allow_pickle = True)

X_trva = np.concatenate((X_tr, X_va), axis = 0)
y_trva = np.concatenate((y_tr, y_va), axis = 0)

np.save("D:/dataset/clip_img_emb_ALL/X_trva.npy", X_trva)
np.save("D:/dataset/clip_img_emb_ALL/X_trva_3.npy", X_trva)

In [ ]:
X_tr= np.load("D:/dataset/clip_img_emb_ALL/X_trva.npy", allow_pickle = True)
y_tr = np.load("D:/dataset/clip_img_emb_ALL/y_trva_3.npy", allow_pickle = True)

X_te = np.load("D:/dataset/clip_img_emb_ALL/X_te.npy", allow_pickle = True)
y_te = np.load("D:/dataset/clip_img_emb_ALL/y_te_3.npy", allow_pickle = True)

In [9]:
cfg = SGDClassifier(
        loss="hinge",
        penalty="l2",
        alpha = 1e-05,
        average = True,
        class_weight = None,
        random_state=42,
        max_iter=1000,
        tol=1e-3,
    )


cfg.fit(X_tr, y_tr)
y_te_pred = cfg.predict(X_te)
macro_f1 = f1_score(y_te, y_te_pred, average="macro")
acc = accuracy_score(y_te, y_te_pred)

print(f"macro-F1 (test): {macro_f1:.4f} | accuracy (test): {acc:.4f}")

macro-F1 (test): 0.4169 | accuracy (test): 0.4447


In [6]:
le = LabelEncoder()
y_tr_enc = le.fit_transform(y_tr)
y_te_enc = le.transform(y_te)


cfgs = [
    GaussianNB(var_smoothing = 1e-06),
    RandomForestClassifier(
        max_depth=12, max_features='sqrt', min_samples_leaf=5, n_estimators=80, n_jobs=-1, random_state=42
    ),
    XGBClassifier(colsample_bytree = 0.5, gamma = 0, learning_rate = 0.1, max_depth= 6, n_estimators= 150, reg_lambda= 1, subsample= 0.8,
        objective="multi:softmax",
        num_class=len(np.unique(y_tr_enc)),
        tree_method="hist", eval_metric="mlogloss",
        n_jobs=-1, random_state=42, verbosity=0
    )
]
for cfg in cfgs:
    print(f"\nConfiguration: {cfg}")

    # XGB requires a numerical target
    if isinstance(cfg, XGBClassifier):
        cfg.fit(X_tr, y_tr_enc)
        y_te_pred = cfg.predict(X_te)
        macro_f1 = f1_score(y_te_enc, y_te_pred, average="macro")
        acc = accuracy_score(y_te_enc, y_te_pred)

    else:
        cfg.fit(X_tr, y_tr)
        y_te_pred = cfg.predict(X_te)
        macro_f1 = f1_score(y_te, y_te_pred, average="macro")
        acc = accuracy_score(y_te, y_te_pred)

    print(f"macro-F1 (train): {macro_f1:.4f} | accuracy (train): {acc:.4f}")


Configuration: GaussianNB(var_smoothing=1e-06)
macro-F1 (train): 0.3986 | accuracy (train): 0.4329

Configuration: RandomForestClassifier(max_depth=12, min_samples_leaf=5, n_estimators=80,
                       n_jobs=-1, random_state=42)
macro-F1 (train): 0.4337 | accuracy (train): 0.4340

Configuration: XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=0,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy